In [1]:
import os
import sys
from pathlib import Path
import pandas as pd
import requests
import duckdb

In [2]:
from finlab import data

In [ ]:
# 引用自建公用模組
sys.path.insert(0, str(Path.cwd().parent))
from proj_util_pkg.settings import ProjEnvSettings

from proj_util_pkg.finlab_api import finlab_manager as flm

## 公用參數設定

In [4]:
# 欄數全展開
pd.set_option("display.max_columns", None)

In [ ]:
# finlab api 服務初始化
finlab = flm.FinlabManager()
data.force_cloud_download = False

## 外部資料讀取  

In [10]:
# 讀取台股收盤價資訊
close = data.get("price:收盤價", save_to_storage=True)
open = data.get("price:開盤價", save_to_storage=True)
high = data.get("price:最高價", save_to_storage=True)
low = data.get("price:最低價", save_to_storage=True)
vol = data.get("price:成交股數", save_to_storage=True)
pe_ratio = data.get('price_earning_ratio:本益比', save_to_storage=True)
pb_ratio = data.get('price_earning_ratio:股價淨值比', save_to_storage=True)
dividend_yield = data.get('price_earning_ratio:殖利率(%)', save_to_storage=True)


In [ ]:
# 收盤價
close.shape

In [ ]:
close

In [6]:
# 設定資料庫路徑
TWSTOCK_DATA_ROOT = os.environ.get("hist_data_path")
twstock_db_path = f"{TWSTOCK_DATA_ROOT}/twstock.duckdb"

In [8]:
# 連線資料庫
conn_duckdb = duckdb.connect(twstock_db_path)

In [ ]:
sample_df = conn_duckdb.execute(f"SELECT * FROM tw_stock_list LIMIT 5").fetch_df()
print(sample_df)

In [ ]:
# 關閉資料庫連線
conn_duckdb.close()
print("資料庫連線已關閉")